In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torchvision
from torchvision.ops import roi_pool
from wsddn import *
from collections import OrderedDict 
from models import *

In [6]:
states = torch.load("../checkpoints/states.pth")

In [5]:
torch.save(states, '../checkpoints/states')

In [ ]:
def init_parameters(module):
    if type(module) in [nn.Conv2d, nn.Linear]:
        torch.nn.init.normal_(module.weight, mean=0.0, std=1e-2)
        torch.nn.init.zeros_(module.bias)

In [ ]:
def copy_parameters(src, target):
    assert src.weight.size() == target.weight.size()
    assert src.bias.size() == target.bias.size()
    src.weight = target.weight
    src.bias = target.bias

In [ ]:
class Combined_Alexnet(nn.Module):
    def __init__(self, K=3, groups=4):
        super(Combined_Alexnet, self).__init__()
        self.K = K
        self.groups = groups
#         alexnet = torchvision.models.alexnet(pretrained=True)
        wsddn_alexnet = WSDDN_Alexnet()
        wsddn_alexnet.load_state_dict(torch.load("../pretrained/eb_2007_wsddn_alexnet.pt"))
        self.pretrained_features = nn.Sequential(*list(wsddn_alexnet.features[:5]._modules.values()))
        self.new_features = nn.Sequential(OrderedDict([
            ('conv3', nn.Conv2d(192, 384, kernel_size=3, stride=1, padding=2, dilation=2)),
            ('relu3', nn.ReLU(inplace=True)),
            ('conv4', nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=2, dilation=2)),
            ('relu4', nn.ReLU(inplace=True)),
            ('conv5', nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=2, dilation=2)),
            ('relu5', nn.ReLU(inplace=True)),
        ]))
        
        copy_parameters(self.new_features.conv3, wsddn_alexnet.features[6])
        copy_parameters(self.new_features.conv4, wsddn_alexnet.features[8])
        copy_parameters(self.new_features.conv5, wsddn_alexnet.features[10])
        
        self.roi_size = (6, 6)
        self.roi_spatial_scale= 0.125
        
        
        self.fc67 = nn.Sequential(*list(wsddn_alexnet.fc67._modules.values()))
        self.fc8c = wsddn_alexnet.fc8c
        self.fc8d = wsddn_alexnet.fc8d
        self.c_softmax = nn.Softmax(dim=1)
        self.d_softmax = nn.Softmax(dim=0)
        for i in range(self.K):
            self.add_module(
                f'refine{i}',
                nn.Sequential(OrderedDict([
#                     (f'groupNorm', nn.GroupNorm(self.groups, 4096)),
                    (f'ic_score{i}', nn.Linear(4096, 21)),
                    (f'ic_probs{i}', nn.Softmax(dim=1))
                ])))
            
    def forward(self, x, regions):
        regions = [regions[0]] # roi_pool require [Tensor(K, 4)]
        R = len(regions[0])
        features = self.new_features(self.pretrained_features(x))
        pool_features = roi_pool(features, regions, self.roi_size, self.roi_spatial_scale).view(R, -1)
        fc7 = self.fc67(pool_features)
        c_score = self.c_softmax(self.fc8c(fc7))
        d_score = self.d_softmax(self.fc8d(fc7))
        proposal_scores = c_score * d_score

        refine_scores = []
        for i in range(self.K):
            refine_scores.append(self._modules[f'refine{i}'](fc7))
        return refine_scores, proposal_scores

In [18]:
model.pretrained_features

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_si

In [ ]:
def read_in_models(model, pretrained_data):
    model.pretrained_features[0].weight.data = pretrained_data['model']['Conv_Body.conv1.0.weight']
    model.pretrained_features[0].bias.data   = pretrained_data['model']['Conv_Body.conv1.0.bias']

    model.pretrained_features[2].weight.data = pretrained_data['model']['Conv_Body.conv1.2.weight']
    model.pretrained_features[2].bias.data   = pretrained_data['model']['Conv_Body.conv1.2.bias']

    model.pretrained_features[5].weight.data = pretrained_data['model']['Conv_Body.conv2.0.weight']
    model.pretrained_features[5].bias.data   = pretrained_data['model']['Conv_Body.conv2.0.bias']

    model.pretrained_features[7].weight.data = pretrained_data['model']['Conv_Body.conv2.2.weight']
    model.pretrained_features[7].bias.data   = pretrained_data['model']['Conv_Body.conv2.2.bias']

    model.pretrained_features[10].weight.data = pretrained_data['model']['Conv_Body.conv3.0.weight']
    model.pretrained_features[10].bias.data   = pretrained_data['model']['Conv_Body.conv3.0.bias']

    model.pretrained_features[12].weight.data = pretrained_data['model']['Conv_Body.conv3.2.weight']
    model.pretrained_features[12].bias.data   = pretrained_data['model']['Conv_Body.conv3.2.bias']

    model.pretrained_features[14].weight.data = pretrained_data['model']['Conv_Body.conv3.4.weight']
    model.pretrained_features[14].bias.data   = pretrained_data['model']['Conv_Body.conv3.4.bias']

    model.pretrained_features[17].weight.data = pretrained_data['model']['Conv_Body.conv4.0.weight']
    model.pretrained_features[17].bias.data   = pretrained_data['model']['Conv_Body.conv4.0.bias']

    model.pretrained_features[19].weight.data = pretrained_data['model']['Conv_Body.conv4.2.weight']
    model.pretrained_features[19].bias.data   = pretrained_data['model']['Conv_Body.conv4.2.bias']

    model.pretrained_features[21].weight.data = pretrained_data['model']['Conv_Body.conv4.4.weight']
    model.pretrained_features[21].bias.data   = pretrained_data['model']['Conv_Body.conv4.4.bias']

    model.new_features[0].weight.data = pretrained_data['model']['Conv_Body.conv5.0.weight']
    model.new_features[0].bias.data   = pretrained_data['model']['Conv_Body.conv5.0.bias']

    model.new_features[2].weight.data = pretrained_data['model']['Conv_Body.conv5.2.weight']
    model.new_features[2].bias.data   = pretrained_data['model']['Conv_Body.conv5.2.bias']

    model.new_features[4].weight.data = pretrained_data['model']['Conv_Body.conv5.4.weight']
    model.new_features[4].bias.data   = pretrained_data['model']['Conv_Body.conv5.4.bias']

    model.fc67[0].weight.data = pretrained_data['model']['Box_Head.fc1.weight']
    model.fc67[0].bias.data   = pretrained_data['model']['Box_Head.fc1.bias']

    model.fc67[3].weight.data = pretrained_data['model']['Box_Head.fc2.weight']
    model.fc67[3].bias.data   = pretrained_data['model']['Box_Head.fc2.bias']

    model.fc8d.weight.data = pretrained_data['model']['Box_MIL_Outs.mil_score0.weight']
    model.fc8d.bias.data   = pretrained_data['model']['Box_MIL_Outs.mil_score0.bias']

    model.fc8c.weight.data = pretrained_data['model']['Box_MIL_Outs.mil_score1.weight']
    model.fc8c.bias.data   = pretrained_data['model']['Box_MIL_Outs.mil_score1.bias']

    model.ic_score1.weight.data = pretrained_data['model']['Box_Refine_Outs.refine_score.0.weight']
    model.ic_score1.bias.data   = pretrained_data['model']['Box_Refine_Outs.refine_score.0.bias']

    model.ic_score2.weight.data = pretrained_data['model']['Box_Refine_Outs.refine_score.1.weight']
    model.ic_score2.bias.data   = pretrained_data['model']['Box_Refine_Outs.refine_score.1.bias']

    model.ic_score3.weight.data = pretrained_data['model']['Box_Refine_Outs.refine_score.2.weight']
    model.ic_score3.bias.data   = pretrained_data['model']['Box_Refine_Outs.refine_score.2.bias']

In [ ]:
pretrained_data = torch.load("../checkpoints/voc2007.pth")

In [ ]:
model.pretrained_features[0].weight.data = pretrained_data['model']['Conv_Body.conv1.0.weight']
model.pretrained_features[0].bias.data   = pretrained_data['model']['Conv_Body.conv1.0.bias']

model.pretrained_features[2].weight.data = pretrained_data['model']['Conv_Body.conv1.2.weight']
model.pretrained_features[2].bias.data   = pretrained_data['model']['Conv_Body.conv1.2.bias']

model.pretrained_features[5].weight.data = pretrained_data['model']['Conv_Body.conv2.0.weight']
model.pretrained_features[5].bias.data   = pretrained_data['model']['Conv_Body.conv2.0.bias']

model.pretrained_features[7].weight.data = pretrained_data['model']['Conv_Body.conv2.2.weight']
model.pretrained_features[7].bias.data   = pretrained_data['model']['Conv_Body.conv2.2.bias']

model.pretrained_features[10].weight.data = pretrained_data['model']['Conv_Body.conv3.0.weight']
model.pretrained_features[10].bias.data   = pretrained_data['model']['Conv_Body.conv3.0.bias']

model.pretrained_features[12].weight.data = pretrained_data['model']['Conv_Body.conv3.2.weight']
model.pretrained_features[12].bias.data   = pretrained_data['model']['Conv_Body.conv3.2.bias']

model.pretrained_features[14].weight.data = pretrained_data['model']['Conv_Body.conv3.4.weight']
model.pretrained_features[14].bias.data   = pretrained_data['model']['Conv_Body.conv3.4.bias']

model.pretrained_features[17].weight.data = pretrained_data['model']['Conv_Body.conv4.0.weight']
model.pretrained_features[17].bias.data   = pretrained_data['model']['Conv_Body.conv4.0.bias']

model.pretrained_features[19].weight.data = pretrained_data['model']['Conv_Body.conv4.2.weight']
model.pretrained_features[19].bias.data   = pretrained_data['model']['Conv_Body.conv4.2.bias']

model.pretrained_features[21].weight.data = pretrained_data['model']['Conv_Body.conv4.4.weight']
model.pretrained_features[21].bias.data   = pretrained_data['model']['Conv_Body.conv4.4.bias']

model.new_features[0].weight.data = pretrained_data['model']['Conv_Body.conv5.0.weight']
model.new_features[0].bias.data   = pretrained_data['model']['Conv_Body.conv5.0.bias']

model.new_features[2].weight.data = pretrained_data['model']['Conv_Body.conv5.2.weight']
model.new_features[2].bias.data   = pretrained_data['model']['Conv_Body.conv5.2.bias']

model.new_features[4].weight.data = pretrained_data['model']['Conv_Body.conv5.4.weight']
model.new_features[4].bias.data   = pretrained_data['model']['Conv_Body.conv5.4.bias']

model.fc67[0].weight.data = pretrained_data['model']['Box_Head.fc1.weight']
model.fc67[0].bias.data   = pretrained_data['model']['Box_Head.fc1.bias']

model.fc67[3].weight.data = pretrained_data['model']['Box_Head.fc2.weight']
model.fc67[3].bias.data   = pretrained_data['model']['Box_Head.fc2.bias']

model.fc8d.weight.data = pretrained_data['model']['Box_MIL_Outs.mil_score0.weight']
model.fc8d.bias.data   = pretrained_data['model']['Box_MIL_Outs.mil_score0.bias']

model.fc8c.weight.data = pretrained_data['model']['Box_MIL_Outs.mil_score1.weight']
model.fc8c.bias.data   = pretrained_data['model']['Box_MIL_Outs.mil_score1.bias']

model.ic_score1.weight.data = pretrained_data['model']['Box_Refine_Outs.refine_score.0.weight']
model.ic_score1.bias.data   = pretrained_data['model']['Box_Refine_Outs.refine_score.0.bias']

model.ic_score2.weight.data = pretrained_data['model']['Box_Refine_Outs.refine_score.1.weight']
model.ic_score2.bias.data   = pretrained_data['model']['Box_Refine_Outs.refine_score.1.bias']

model.ic_score3.weight.data = pretrained_data['model']['Box_Refine_Outs.refine_score.2.weight']
model.ic_score3.bias.data   = pretrained_data['model']['Box_Refine_Outs.refine_score.2.bias']

In [ ]:
for k, v in pretrained_data['model'].items():
    print(k)

In [7]:
model = Combined_VGG16()

In [11]:
for k, v in states.items():
    print(k)

base.features.0.weight
base.features.0.bias
base.features.2.weight
base.features.2.bias
base.features.5.weight
base.features.5.bias
base.features.7.weight
base.features.7.bias
base.features.10.weight
base.features.10.bias
base.features.12.weight
base.features.12.bias
base.features.14.weight
base.features.14.bias
base.features.17.weight
base.features.17.bias
base.features.19.weight
base.features.19.bias
base.features.21.weight
base.features.21.bias
base.features.24.weight
base.features.24.bias
base.features.26.weight
base.features.26.bias
base.features.28.weight
base.features.28.bias
base.classifier.0.weight
base.classifier.0.bias
base.classifier.3.weight
base.classifier.3.bias
base.classifier.6.weight
base.classifier.6.bias
features.0.weight
features.0.bias
features.2.weight
features.2.bias
features.5.weight
features.5.bias
features.7.weight
features.7.bias
features.10.weight
features.10.bias
features.12.weight
features.12.bias
features.14.weight
features.14.bias
features.17.weight
fea

In [8]:
model.pretrained_features[0]

Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

In [ ]:
model.pretrained_features[0].weight.data = states['model']['Conv_Body.conv1.0.weight']
model.pretrained_features[0].bias.data   = states['model']['Conv_Body.conv1.0.bias']

In [ ]:
model.new_features.conv5_1.weight

In [ ]:
vgg = torchvision.models.vgg16(pretrained=True)

In [ ]:
vgg.features[24].bias == model.new_features.conv5_1.bias

In [ ]:
def read_in_states(model, states):
    model.pretrained_features[0].weight.data = states['features.0.weight']
    model.pretrained_features[0].bias.data   = states['features.0.bias']

    model.pretrained_features[2].weight.data = states['features.2.weight']
    model.pretrained_features[2].bias.data   = states['features.2.bias']

    model.pretrained_features[5].weight.data = states['features.5.weight']
    model.pretrained_features[5].bias.data   = states['features.5.bias']

    model.pretrained_features[7].weight.data = states['features.7.weight']
    model.pretrained_features[7].bias.data   = states['features.7.bias']

    model.pretrained_features[10].weight.data = states['features.10.weight']
    model.pretrained_features[10].bias.data   = states['features.10.bias']

    model.pretrained_features[12].weight.data = states['features.12.weight']
    model.pretrained_features[12].bias.data   = states['features.12.bias']

    model.pretrained_features[14].weight.data = states['features.14.weight']
    model.pretrained_features[14].bias.data   = states['features.14.bias']

    model.pretrained_features[17].weight.data = states['features.17.weight']
    model.pretrained_features[17].bias.data   = states['features.17.bias']

    model.pretrained_features[19].weight.data = states['features.19.weight']
    model.pretrained_features[19].bias.data   = states['features.19.bias']

    model.pretrained_features[21].weight.data = states['features.21.weight']
    model.pretrained_features[21].bias.data   = states['features.21.bias']

    model.new_features[0].weight.data = states['features.24.weight']
    model.new_features[0].bias.data   = states['features.24.bias']

    model.new_features[2].weight.data = states['features.26.weight']
    model.new_features[2].bias.data   = states['features.26.bias']

    model.new_features[4].weight.data = states['features.28.weight']
    model.new_features[4].bias.data   = states['features.28.bias']

    model.fc67[0].weight.data = states['fcs.0.weight']
    model.fc67[0].bias.data   = states['fcs.0.bias']

    model.fc67[3].weight.data = states['fcs.3.weight']
    model.fc67[3].bias.data   = states['fcs.3.bias']

    model.fc8d.weight.data = states['fc_c.weight']
    model.fc8d.bias.data   = states['fc_c.bias']

    model.fc8c.weight.data = states['fc_d.weight']
    model.fc8c.bias.data   = states['fc_d.bias']

    model.ic_score1.weight.data = states['fc_ic_score.weight']
    model.ic_score1.bias.data   = states['fc_ic_score.bias']

    model.ic_score2.weight.data = states['fc_ic_score1.weight']
    model.ic_score2.bias.data   = states['fc_ic_score1.bias']

    model.ic_score3.weight.data = states['fc_ic_score2.weight']
    model.ic_score3.bias.data   = states['fc_ic_score2.bias']

In [14]:
model.pretrained_features[0].weight.data = states['features.0.weight']
model.pretrained_features[0].bias.data   = states['features.0.bias']

model.pretrained_features[2].weight.data = states['features.2.weight']
model.pretrained_features[2].bias.data   = states['features.2.bias']

model.pretrained_features[5].weight.data = states['features.5.weight']
model.pretrained_features[5].bias.data   = states['features.5.bias']

model.pretrained_features[7].weight.data = states['features.7.weight']
model.pretrained_features[7].bias.data   = states['features.7.bias']

model.pretrained_features[10].weight.data = states['features.10.weight']
model.pretrained_features[10].bias.data   = states['features.10.bias']

model.pretrained_features[12].weight.data = states['features.12.weight']
model.pretrained_features[12].bias.data   = states['features.12.bias']

model.pretrained_features[14].weight.data = states['features.14.weight']
model.pretrained_features[14].bias.data   = states['features.14.bias']

model.pretrained_features[17].weight.data = states['features.17.weight']
model.pretrained_features[17].bias.data   = states['features.17.bias']

model.pretrained_features[19].weight.data = states['features.19.weight']
model.pretrained_features[19].bias.data   = states['features.19.bias']

model.pretrained_features[21].weight.data = states['features.21.weight']
model.pretrained_features[21].bias.data   = states['features.21.bias']

model.new_features[0].weight.data = states['features.24.weight']
model.new_features[0].bias.data   = states['features.24.bias']

model.new_features[2].weight.data = states['features.26.weight']
model.new_features[2].bias.data   = states['features.26.bias']

model.new_features[4].weight.data = states['features.28.weight']
model.new_features[4].bias.data   = states['features.28.bias']

model.fc67[0].weight.data = states['fcs.0.weight']
model.fc67[0].bias.data   = states['fcs.0.bias']

model.fc67[3].weight.data = states['fcs.3.weight']
model.fc67[3].bias.data   = states['fcs.3.bias']

model.fc8d.weight.data = states['fc_c.weight']
model.fc8d.bias.data   = states['fc_c.bias']

model.fc8c.weight.data = states['fc_d.weight']
model.fc8c.bias.data   = states['fc_d.bias']

model.ic_score1.weight.data = states['fc_ic_score.weight']
model.ic_score1.bias.data   = states['fc_ic_score.bias']

model.ic_score2.weight.data = states['fc_ic_score1.weight']
model.ic_score2.bias.data   = states['fc_ic_score1.bias']

model.ic_score3.weight.data = states['fc_ic_score2.weight']
model.ic_score3.bias.data   = states['fc_ic_score2.bias']

In [18]:
model.pretrained_features[0].weight

Parameter containing:
tensor([[[[-0.5502,  0.1449,  0.5306],
          [-0.5797,  0.3582,  0.7667],
          [-0.6870, -0.0460,  0.4855]],

         [[ 0.1760,  0.0104, -0.0808],
          [ 0.0447, -0.0699, -0.2597],
          [ 0.1326, -0.1725, -0.1319]],

         [[ 0.3114, -0.1670, -0.4281],
          [ 0.4733, -0.0840, -0.4877],
          [ 0.6297,  0.0177, -0.2785]]],


        [[[ 0.2331,  0.1272,  0.1865],
          [-0.4269, -0.2426,  0.2467],
          [-0.2499,  0.1421, -0.0047]],

         [[-0.1407, -0.2188,  0.1504],
          [-0.8404, -0.3513,  0.5636],
          [-0.2419,  0.5187,  0.5389]],

         [[-0.3139, -0.3699, -0.1305],
          [-0.4708, -0.1547,  0.3459],
          [ 0.0543,  0.5863,  0.4958]]],


        [[[ 0.1773,  0.5212,  0.0102],
          [-0.2708, -0.7154,  0.3127],
          [-0.0754, -0.2203,  0.3343]],

         [[ 0.3078,  0.6690,  0.0198],
          [-0.4660, -1.0686,  0.3337],
          [-0.0811, -0.3057,  0.5430]],

         [[ 0.3144,  0